In [ ]:
import pandas as pd
import hashlib
from collections import Counter

In [ ]:
FILENAME = "data/Portugal.txt"

In [ ]:
header = ["phone", "id2", "firstname", "lastname", "gender", "city1", "city2", "status", "work", "date01", "date02", "date03", "date04", "date05"]
dtype = {"date01": object, "date02": object, "date03":object, "phone": object}
dirty_df = pd.read_csv(FILENAME, sep=":", names=header, dtype=dtype)
dirty_df

In [ ]:
df = dirty_df[["phone", "id2", "gender", "city1", "city2", "status", "work"]].copy()
df["date"] = pd.to_datetime(dirty_df["date01"].astype(str) + ":" + dirty_df["date02"].astype(str) + ":" + dirty_df["date03"].astype(str), errors='ignore')
df["name"] = dirty_df["firstname"].str.lower()+ " " + dirty_df["lastname"].str.lower()
df.head()

In [ ]:
SEARCH_NAME = "António Costa"
search_term = ".*".join(SEARCH_NAME.lower().split(" "))
print(search_term)
matches = df[df["name"].str.contains(search_term, regex=True, na=False)]
matches

In [ ]:
def _hash(v):
    m = hashlib.sha256()
    m.update(bytes(v, encoding="utf-8"))
#     print(len(m.hexdigest())-16) # used in JS substring(from)
    return m.hexdigest()[-16:]

In [ ]:
df["phone_hash"] = df.phone.map(_hash)
df.head()

In [ ]:
assert Counter(df["phone_hash"]).most_common()[0][1]==1, "collision with the hashes, increase chars used in '_hash()' method"

with open("../docs/hashed_data.json", 'w', encoding='utf-8') as outf:
    df[["phone_hash", "name"]].set_index("phone_hash").to_json(outf, force_ascii=False, orient='columns')